<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/20_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install hdwallet py_crypto_hd_wallet cryptos eciespy qrcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.1 MB/s 
     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 69 kB 5.6 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 62 kB 283 kB/s 
     |████████████████████████████████| 119 kB 40.0 MB/s 
     |████████████████████████████████| 829 kB 39.2 MB/s 
     |████████████████████████████████| 305 kB 45.1 MB/s 
     |████████████████████████████████| 1.3 MB 29.7 MB/s 
     |████████████████████████████████| 871 kB 17.9 MB/s 
     |████████████████████████████████| 2.3 MB 37.0 MB/s 
     |████████████████████████████████| 856 kB 58.2 MB/s 
     |████████████████████████████████| 1.0 MB 63.2 MB/s 
     |████████████████████████████████| 105 kB 39.9 MB/s 
     |████████████████████████████████| 481 kB 48.4 MB/s 
  Created wheel for pysha3: filename=pysh

# Our Key Handing Up Until Now

Real drawbacks:

- To send and recieve coins we would have to create a new private key, public key and address. 
- If we wanted to send coins from one address to another we'd have to create another private key, public key and address.
- If we wanted to send to one another we'd have to send each other our addresses.
- If we wanted to create a multisig wallet together we'd have to share our public keys.

Wouldn't it be nice if we could generate a single private key from which we could generate a whole set of private keys in a deterministic fashion?

Wouldn't it be nice if we could share a single public key (associate with the above single private key) from which we could generate the whole set of associated public keys in a deterministic fashion?

**This better world is possible with hierarchical deterministic wallets (HD wallets)**


Hierarchical deterministic wallets, also know as HD-wallets, are the standard in the cryptocurrency world but applicable for any application where we new to generate and share keys in this fashion.  We will learn how to use them appropriately.

https://pypi.org/project/py-crypto-hd-wallet/



# BIP44 Derivation Paths

| Coin | Symbol | Derivation Path |
| :------ | :----: | ----------:|
|**Bitcoin**| BTC |`m/44'/0'/0'/0/0`|
|**Litecoin**| LTC |`m/44'/2'/0'/0/0`|
|**Dogecoin**| DOGE |`m/44'/3'/0'/0/0`|
|**DigiByte**| DGB |`m/44'/20'/0'/0/0`|
|**Zcash**| ZEC |`m/44'/133'/0'/0/0`|
|**BitcoinCash**| BCH |`m/44'/145'/0'/0/0`|
|**BitcoinSV**| BSV | `m/44'/236'/0'/0/0`|

Bitcoin Improvement Proposal (BIP) 44 defines the standard derivation path for wallets which generate Pay-to-Public-Key-Hash (P2PKH) addresses. BIP 44 also defines the prefixes to be used with associated extended keys.

According to BIP 44, wallets which generate P2PKH addresses should use a derivation path beginning with `m/44'/`. This means the first address generated by a mainnet Bitcoin wallet will have a derivation path of `m/44'/0'/0'/0/0`.



`m/44'/0'/0'/0/0` is labeled as `master / purpose / coin_type / account / change / address_index`

- **master** all HD wallets begin with `m` in the first position
- **purpose** all coins adoptin BIP44  use `44`
- **coin_type** is used to differentiate between different coins 
- **account** is used to have a branch point for different subsets of key pairs
- **change** this field is used to differentiate between receipt addresses and change addresses
- **address_index** this final field is used to iterate through different keypairs/addresses

<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/hd_wallet.png"  width="500"/>
</div>

The brilliant feature of this structure is that there is a hierarchy of public and private keys.

With ONE master private key you can control/generate many private keys for many different coins.  In addition you can share ONE public key to share ALL the derived public keys and addresses.

Say you have a business with a point-of-sale cash register that collects DOGE transactions. Each transaction can have a seperate address. You need only share the public key with the `m/44'/3'/0'/0` derivation path. The POS software can then derive the addresses for `m/44'/3'/0'/0/0`, `m/44'/3'/0'/0/1`, `m/44'/0'/0'/0/2`...etc

For our purposes...all the students can generate a single master private key and share one public key with hq. HQ can then access an array of derived public keys and the associated crypto addresses.

# Mnemonic Seed Phrases

BIP39 is the use of a mnemonic phrase -- a group of easy to remember words -- to serve as a back up to recover your wallet and coins in the event your wallet becomes compromised, lost, or destroyed. This is also known as a mnemonic seed (phrase), recovery phrase, wallet back up, master seed, etc. 

<div>
<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/mnemonic.png" width="400"/>
</div>

These words aren't just any words. They are pulled from a specific list of 2048 words known as the BIP39 wordlist. Upon start up, wallets that utilize the BIP39 standard will provide you a 12-24 word phrase randomly chosen from the standard BIP39 wordlist.

Mnemonic seeds are a way of storing the root private key in a human readable format.

The `hdwallet.utils.generate_mnemonic()` function uses a two arguments, strength and language. 

- **strength** 128, 160, 192, 224 or 256
- **language** english, french, italian, spanish, chinese_simplified, chinese_traditional, japanese or korean

In [2]:
from hdwallet.utils import generate_mnemonic
from py_crypto_hd_wallet import HdWalletBipFactory, HdWalletBip44Coins, HdWalletBipChanges
import psutil
import os
import pandas as pd
import ecies
import eth_keys

In [3]:
secret_phrase=generate_mnemonic(language="english", strength=256)
secret_phrase

'one cave album require steak empower scheme shine ride runway bright solve afraid swift enable club depend more involve cradle estate style guide sleep'

In [4]:
hd_wallet_factory = HdWalletBipFactory(HdWalletBip44Coins.DOGECOIN)

In [5]:
my_doge_wallet = hd_wallet_factory.CreateFromMnemonic('DD_wallet', secret_phrase)

In [6]:
my_doge_wallet.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 4)

In [7]:
my_doge_wallet_dict=my_doge_wallet.ToDict()
my_doge_wallet_dict

{'account_idx': 0,
 'account_key': {'address': 'DABjC3UrVho9QcR4K6wKFpaeobRLCeLxXL',
  'ex_priv': 'dgpv57arGcvU6WZ2UAp14uCUfMf5VsAfTsHirkBV2bcUF1utCFec5cWg5GyeJVKdodpD4kCoeGTWnXeGuKAiLzW5dvBex6oKMh9q8ZgX4G6AVm2',
  'ex_pub': 'dgub8rTscJrMefDMm2UA78pSnDCfxRYJHzKS9LxhBi1T38xtkJmJVP4SiaPHshJmZavqw9euxmXcR7d4ghyXcuyYAxDyQVZPivEED1TfwDpxXzT',
  'raw_compr_pub': '0265ed8348284e2feddb4b91255913a7cb09e6ca7253b59200a5af57c5117f5669',
  'raw_priv': 'e24d80be006f37b49843009f31ccfe88188631623c2aa7eabff558ff622d85ab',
  'raw_uncompr_pub': '0465ed8348284e2feddb4b91255913a7cb09e6ca7253b59200a5af57c5117f56698897f4b0863378692722fe610550ce2bbfece6f560a2786521d7daec996099cc',
  'wif_priv': 'QWCXi5NjSC6SHFyKc1nH2oK3pQX6vXyCqeGuCYxpG51iA3GQ1QFy'},
 'address': {'address_0': {'address': 'DGya8kKyo1Lm1RUo1g6sJHo31v2w48sA5Q',
   'ex_priv': 'dgpv5BdudMp6RWUPrC7N7z778Q6T91zpbw6B7cxb1geCtjNDMvoqjCkqkp9H788yATusxXuEwykPNeHFP95kNBBZV7KwysjVaKbwHnqeEe25hY6',
   'ex_pub': 'dgub8vWvy3jyyf8j93mXADj5FFe3baNTS47tQDjoAo3B

In [8]:
my_doge_wallet_dict.keys()

dict_keys(['wallet_name', 'spec_name', 'coin_name', 'mnemonic', 'passphrase', 'seed_bytes', 'master_key', 'purpose_key', 'coin_key', 'account_idx', 'account_key', 'change_idx', 'change_key', 'address_off', 'address'])

In [9]:
my_doge_wallet_dict['address']['address_1']['wif_priv'] #this is the private key to address 1

'QUGpjZXTJFDT76g9dAwVq3p3cQnVDZnZsVLoxexHtP4DXXu352by'

In [10]:
my_doge_wallet_dict['address']['address_1']['raw_uncompr_pub'] #this is the public key to address 1

'04c0f24d1658389818cc5c99e8b2f6c6f39ac7f5b1c01c2fc6e8a1d3d8e7de48e4e4e606076ed3417e340e4f974105167db57cf406dbaf7748577d015e8cd6eb75'

Remember the hierarchy is `master / purpose / coin_type / account / change / address_index`. Each of these nodes has it's own xpub hierarchical public key. Sharing a xpub gives all of the child public keys. In order to share all of the external addresses (not internal change adresses) we share the xpub associated with the `change` node.

Sharing the xpub from a higher node would reveal more public keys. Sharing at the `account` level would reveal the public keys for receive addresses and change addresses (external and internal).

In [11]:
my_doge_wallet_dict['change_key']['ex_pub'] #this is the xpub key that can be used to generate all the public keys 

'dgub8tTKRWzJ75BGLVrhJSzbgGkJtkAxv71scdBAmP74mkFJSezwSCgfAAQUfGKSYQCteThMWttyfy62XPYndYMr2XznbQXbMcd84BtmuMcJqH4'

In [12]:
def get_addresses(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['address'] for addy in addys]

def get_privates(wallet,form='wif'):
    addys=wallet.ToDict()['address']
    if form=='wif':
      return [ addys[addy]['wif_priv'] for addy in addys]
    else:
      return [ addys[addy]['raw_priv'] for addy in addys]
def get_publics(wallet):
    addys=wallet.ToDict()['address']
    return [ addys[addy]['raw_uncompr_pub'] for addy in addys]

In [13]:
get_addresses(my_doge_wallet)

['DGya8kKyo1Lm1RUo1g6sJHo31v2w48sA5Q',
 'DTMpsQwuFohWLQtQ7MNiAoZfpJQm2qgNMN',
 'DR9WACjXTK2MFUxcC35Hkzmaz9Jb2vijSb',
 'D8Wr8FFEJ3XUX3hAhkseBDBNAxtRG4RwW3']

# File With Some Ex_PubKeys

```text
student,ex_pubkey
00,dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm
01,dgub8tiZR8EbP42Rg4EAn7jYYQRLjCZ2UHg5Bwdb4eASRBvNYwanrTdp8jvN2bHdakaevMj82u9tknJaYvsiaSMpijn7aP2hSsJbLbj8qbuBmZU
02,dgub8tTFHkG1FajRZH2nscUV9166uRMRRAvbeh6zXCctUVgpNwvWpZ16JT1WrSwW46p2hvfPZNCKmb5xQBcUwKBip7iM3tUNMzAkqDDysR1UhbE
03,dgub8tZWJSrqUJ9KDRAQEHU4YH9ToP6f16jLsvi1cP4kA45MRve9PnBM3BPUgPshBoJBS1BebakEw5Qh9W2pAfTWV8YVgSeUfmyjCkScMGabGMK
04,dgub8tP4gsyL1R7U5eZVTDZbA5DF4foatJGq9EikgtaX3rtUp5hZ1QT9CnCxhf1GmceVsH8xjJmzcCMyXdSxekjvQ8M5fZKaNoenMenPDDmPfwM
05,dgub8uqjRZAXa5dvQ2wTaaxTeS45EDd3BbNCo6X1ifCnH2AJvRiH3CsdS3yTqFMvfU3EptHTtDVUb2V7pb5YNaecf5xPkUQ16CWxWpbPLAxo7hL
06,dgub8uTT3NrNwnwMW1pYo4AHR4XSEUf4pZQtHjg1xSuxSBZCvvketHKK3gKcLuiu8BNW9LTgQ5oZs35PT6ToR6xr4Q2PCqfvvoxQ18HSsS2fPcw


09,dgub8seQT7BUFMQfoVNgMG7W9tXhDcLEYxyM6Ntf4k5qN49cMaeRK1iZegYCpQ4RGeEMUxBtFFvhpWqvvk3McuEUojdhPvYy3wWmXPP4u89RcmN
```

The 00 key and 09 key are xpubs to the same wallet. 00 is from change=0 node while 09 is from account=0 node

In [14]:
ex_00='dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm'

In [15]:
watch_00=hd_wallet_factory.CreateFromExtendedKey("watcher_00",ex_00)

In [16]:
watch_00.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 10)

In [17]:
watch_00.IsWatchOnly()

True

In [18]:
watch_00.ToDict()

{'address': {'address_0': {'address': 'DBcNgj9T7zy8b12igfRPQw7XbnaptwqCbm',
   'ex_pub': 'dgub8w6D9esvEam9y5RLQDUvZ8ie96mxQYHeDpq9kBbzKtpT5vcrmCh7BQq1KEJ1aW7m85qQdnynLNxjgd7J3dRG2ZAfWAupuGDKrx5EQTsQKVH',
   'raw_compr_pub': '03797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc7',
   'raw_uncompr_pub': '04797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc74f41335fc8ae94be4d55daaf3afed7af3057dcb44aede7ca9d9888a02ed62da9'},
  'address_1': {'address': 'DTAvSJgxbQ7hQSffTB5tg9MPZzJ6VKLQGJ',
   'ex_pub': 'dgub8w6D9esvEamA1HhaMSogPDau7oWhnThpS2phDqstSoJJKdUD5iZRQXPZSeP2mM2gkVxeSadkUSeqhN98HDYBEziAU2oad46F4osv58MnFwZ',
   'raw_compr_pub': '03bf03dea1cb1dddb81f968d3828e6da8f6e34447c507c6add66b54f59551a2afb',
   'raw_uncompr_pub': '04bf03dea1cb1dddb81f968d3828e6da8f6e34447c507c6add66b54f59551a2afbb3c676fb42d6e71ea3073a006d6073bb023de3d987fa4cb4373281fb9e8aac8f'},
  'address_2': {'address': 'DJvdop5eker2ZHij9Gmh1qGCc7G2TijPy3',
   'ex_pub': 'dgub8w6D9esvEamA5TFkyAF4Yh5ixf

In [56]:
get_addresses(watch_00)

['DBcNgj9T7zy8b12igfRPQw7XbnaptwqCbm',
 'DTAvSJgxbQ7hQSffTB5tg9MPZzJ6VKLQGJ',
 'DJvdop5eker2ZHij9Gmh1qGCc7G2TijPy3',
 'DRsTTHnf9SYiMt8vinTfNDi78Ag55te62e',
 'DPjudXU9XHz487RpXcAfSPbY4h1F85cyzd',
 'DJuRujqNU6yYRx7TUPhdSBomnyjq3fJvUF',
 'D8XbpiDauFQftCbf1gy8FTUHKQKm9KcfuX',
 'D8VhDPNQzsHCqrLkPWERVJyxT4SoLZNSzL',
 'DFb5pALcHZrTtFdRVKma8rWM73XZADQWRN',
 'DDpZQ8uPNXMrcADuY2fphnHgYeVwpD5CA2']

This xpub number 9 is from the account 0 node, therefore both internal and external addresses are available

In [ ]:
ex_09='dgub8seQT7BUFMQfoVNgMG7W9tXhDcLEYxyM6Ntf4k5qN49cMaeRK1iZegYCpQ4RGeEMUxBtFFvhpWqvvk3McuEUojdhPvYy3wWmXPP4u89RcmN'
watch_09_ext=hd_wallet_factory.CreateFromExtendedKey("watcher_09",ex_09)
watch_09_int=hd_wallet_factory.CreateFromExtendedKey("watcher_09",ex_09)
watch_09_ext.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_EXT, addr_num = 4)
watch_09_int.Generate(account_idx = 0, change_idx = HdWalletBipChanges.CHAIN_INT, addr_num = 4)
print(get_addresses(watch_09_ext))
print('-'*50)
print(get_addresses(watch_09_int))

#Identity


Identity has different compnents and posible constructions. Here we will form identity as a json dictionary. Every key value pair is opt-in. 

```python
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'HexSignatutureOfTheTransactionThatStoresThis'} }
```


```python
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'image' : 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d',
                  'addr_0' : 'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro'
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'} }
```

There can be four types of entries:
- declaritive entries: these are names or datapoints which are stated unproven as self identifiers
- social pointers: these are pointers to social media or websites that contain references to the identity transaction. these references serve as proof in the identity handshake.
- public keys: these are references to hd keys or single keys. these entries are dictionaries that contain the a public key reference and a signature from that keypair of the identity transaction
- prev_id: this is a previous identity transaction hash 

There should be different types of trust/proofs implemented. Maybe and account with both keys capable of withdraw. This method of identity construction does allow for redefined personal digital boundaries.

We can also reference other identity transactions. The authenticated dictionary should be constructed publickeys first, then previous identity transactions, then finally declaratives and socials.

# Header

the header is `\xc1\xdd\x00\x01\x1d\x00\x00`

`1d` is the byte for Identity inscriptions

In [20]:
identity_dict = { 'name' : 'DrDoeg',
                  'field' : 'Physics', 
                  'twitter' : 't_physics',
                  'image' : 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d',
                  'addr_0' : {'addr':'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro',
                              'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'},
                  'hd_pub_0' : {'pub/44/3/0/0':'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
                                 'sig_trans':'d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'} }

identity_dict       

{'addr_0': {'addr': 'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro',
  'sig_trans': 'd1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'},
 'field': 'Physics',
 'hd_pub_0': {'pub/44/3/0/0': 'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
  'sig_trans': 'd1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101'},
 'image': 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d',
 'name': 'DrDoeg',
 'twitter': 't_physics'}

In [23]:
import json

pack=json.dumps(identity_dict, indent = '\t').encode()
print(len(pack)/80)
print(pack.decode())

8.225
{
	"name": "DrDoeg",
	"field": "Physics",
	"twitter": "t_physics",
	"image": "aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d",
	"addr_0": {
		"addr": "DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro",
		"sig_trans": "d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101"
	},
	"hd_pub_0": {
		"pub/44/3/0/0": "dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm",
		"sig_trans": "d1ec81ffae055329a23787dc0d9e1525af8893ef3f00a7568505f6220a9b82de4147da07d2a00552a12406c708c571869e0bc892266f9a13dc15528b491cc8f101"
	}
}


This is the identity transaction id:

`1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299`

In [24]:
identity_txn='1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299'

In [25]:
(hdr,bdy)=read_cadenas('1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299')

In [26]:
id_dct=json.loads(bdy.decode())
id_dct

{'addr_0': {'addr': 'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro',
  'sig_trans': '3211a4fe8361c35fbd83c243010c73ca4872e2f4646e5ed3042ff5708797ca0a777760ae2e2ca4a87bc9fb0bccb45f7ed5779cfff93814d38d7aa9e06977aac801'},
 'field': 'Physics',
 'hd_pub_0': {'pub/44/3/0/0': 'dgub8uu2ChGGLQcFXt5QEBoHJNHhxubH6LXWxzMcVKNRfMyQgCfpWquMYkNsiXsQeEddLzX692Am7rgZ7s7wLqts1FbXNuA2pGtRdPuht92J4pm',
  'sig_trans': '406f42781965a23fa10fa547db1861f1fe85c3fb9e76ad3e1132e0d33e7d8b0111375b062e4edc754032c7dedaa6fc135dae131d1639be2c9e19f5423dc6712301'},
 'image': 'aa0c3ea6b38b2238725df2457eb328bbebd7d91ac5241c24de8ea11616c2bb6d',
 'name': 'DrDoeg',
 'twitter': 't_physics'}

In [27]:
hdr

'c1dd00011d00007c4465636c61726174696f6e206f66204964656e746974797c'

In [28]:
utterance=b'\x1d\x00\x00'+bytes.fromhex('1566b4b36e938e4dab05718d16d1d3261ad7b40a2ea4ecaa34b52a822bbe4299')
utterance

b'\x1d\x00\x00\x15f\xb4\xb3n\x93\x8eM\xab\x05q\x8d\x16\xd1\xd3&\x1a\xd7\xb4\n.\xa4\xec\xaa4\xb5*\x82+\xbeB\x99'

In [32]:
import eth_keys
signature_addr_0=eth_keys.datatypes.Signature(bytes.fromhex('3211a4fe8361c35fbd83c243010c73ca4872e2f4646e5ed3042ff5708797ca0a777760ae2e2ca4a87bc9fb0bccb45f7ed5779cfff93814d38d7aa9e06977aac801'))
signature_hdpub_0=eth_keys.datatypes.Signature(bytes.fromhex('406f42781965a23fa10fa547db1861f1fe85c3fb9e76ad3e1132e0d33e7d8b0111375b062e4edc754032c7dedaa6fc135dae131d1639be2c9e19f5423dc6712301'))

In [45]:
import cryptos 
doge=cryptos.Doge()
doge.pubtoaddr('04'+signature_addr_0.recover_public_key_from_msg(utterance).to_hex()[2:])

'DRdoEgDC7naAn9bsXEHqdCCPaYN7S8i5ro'

In [47]:
doge.pubtoaddr('04'+signature_hdpub_0.recover_public_key_from_msg(utterance).to_hex()[2:])

'DBsM3r6RiTX8PAKfyzasNdBZUCSEz52nJf'

In [48]:
'04'+signature_hdpub_0.recover_public_key_from_msg(utterance).to_hex()[2:]

'04797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc74f41335fc8ae94be4d55daaf3afed7af3057dcb44aede7ca9d9888a02ed62da9'

In [ ]:
'04797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc74f41335fc8ae94be4d55daaf3afed7af3057dcb44aede7ca9d9888a02ed62da9'

In [55]:
doge.pubtoaddr('04797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc74f41335fc8ae94be4d55daaf3afed7af3057dcb44aede7ca9d9888a02ed62da9')

'DBsM3r6RiTX8PAKfyzasNdBZUCSEz52nJf'

In [49]:
doge.pubtoaddr('03797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc7')

'DBcNgj9T7zy8b12igfRPQw7XbnaptwqCbm'

In [53]:
doge.pubtoaddr('03797a01b3e9c7ac5273ab0ff37c0917dfe0f7909ec2144977aad5f2d96bae1fc7')

'DBcNgj9T7zy8b12igfRPQw7XbnaptwqCbm'